# DA- VAE, Diffusion (MDD,BP_dataset)

* Using Keras/Tensorflow

In [ ]:
import os
import time
# import shap ## for XAI
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
# import pingouin as pg
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [ ]:
from numpy import dot
from numpy.linalg import norm

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers import Dense , Activation, Dropout, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.callbacks import Callback

from scipy.special import rel_entr
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, train_test_split, ParameterGrid
from sklearn import decomposition, metrics
from sklearn.cluster import KMeans
from sklearn.metrics import cohen_kappa_score,f1_score, confusion_matrix
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.manifold import TSNE

from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Lambda, Conv1D, Flatten, Reshape, UpSampling1D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.losses import mse, MeanSquaredError

## Original data check (MDD,BP)

In [ ]:
### BP, MDD research questions dataset
data_ori = pd.read_csv('E:/RESEARCH/Datasets/BIPOLAR_MWJ/data_andrew_only_vars_missing_handling_.csv')

In [ ]:
### data shape, variables check
print("The shape of the original MDD+BP dataset is:", data_ori.shape)

In [ ]:
data_ori['target_info'] = data_ori['target_info'].replace({0: 'control'})
data_ori['target_info'] = data_ori['target_info'].replace({1: 'mdd'})
data_ori['target_info'] = data_ori['target_info'].replace({2: 'bpi'})
data_ori['target_info'] = data_ori['target_info'].replace({3: 'bpii'})

In [ ]:
data_ori["target_info"].value_counts()

> There are some dataset columns with weird data. \
> For example, section 6-2 should not contain the data with 0 value, but they exist.

In [ ]:
# 특정 세션의 열들 선택 (0이 나올 수 없는 애들)
s2_1_columns = [col for col in data_ori.columns if 's2_1' in col]
s6_2_columns = [col for col in data_ori.columns if 's6_2' in col]

In [ ]:
# 'S2'를 포함한 열들에서 값이 0인 값을 NA로 바꾸기
data_ori[s2_1_columns] = data_ori[s2_1_columns].replace(0, np.nan)
data_ori[s6_2_columns] = data_ori[s6_2_columns].replace(0, np.nan)

In [ ]:
print("Subject dataset that has no target information is:", data_ori['target_info'].isna().sum())

* Drop only rows with NA in target_info column

In [ ]:
data_ori = data_ori.dropna(subset = ['target_info'], how='any', axis=0)

In [ ]:
data_ori.shape

In [ ]:
data_ori.columns

* Handling dataset without section 8

In [ ]:
data_no_s8 = data_ori.drop(data_ori.columns[data_ori.columns.str.startswith('s8')], axis=1)

In [ ]:
data_fill_s8 = data_ori
for column in data_fill_s8.columns:
    if 's8' in column:
        data_fill_s8[column] = data_fill_s8[column].fillna(0)

In [ ]:
data_s246 = data_fill_s8.iloc[:, (data_fill_s8.columns.str.contains(('s2|s4|s6|info')))]
data_s2467 = data_fill_s8.iloc[:, (data_fill_s8.columns.str.contains(('s2|s4|s6|s7|info')))]

### Missing data imputation using same label mean

In [ ]:
data_fill_mean = data_fill_s8.copy()
# data_fill_mean = data_fill_mean.drop(['subject_info', 'birth_info', 'sex_info', 'age_info', 'family_hx_info'], axis=1)

In [ ]:
## Function for filling the row containing more than 10 missing dataset with target group means
def fill_missing_with_mean(df):
    rows_with_many_nans = df[df.isnull().sum(axis=1) >= 5] ## row with more than 10 missing data

    for idx, row in rows_with_many_nans.iterrows():
        target_value = row['target_info'] ## get the target_info of each row
        group_rows = df[(df['target_info'] == target_value) & (df.index != idx)]
        
        if not group_rows.empty: ## if there is more than one same target_info row
            mean_values = group_rows.select_dtypes(include=[np.number]).mean()
            for col in df.columns:
                if pd.isnull(df.at[idx, col]) and col in mean_values:
                    df.at[idx, col] = mean_values[col]

    return df

In [ ]:
data_fill_mean = fill_missing_with_mean(data_fill_mean)

### Data selection (for specific group)

In [ ]:
data_disorder = data_fill_mean.copy()

In [ ]:
data_control = data_disorder[data_disorder['target_info']=="control"]
data_mdd = data_disorder[data_disorder['target_info']=="mdd"]
data_bpi = data_disorder[data_disorder['target_info']=="bpi"]
data_bpii = data_disorder[data_disorder['target_info']=="bpii"]
# data_bp_only = data_bp_only.drop(mdd)

In [ ]:
data_control.shape

In [ ]:
print("The size of control group dataset is:", data_control.shape)
print("The size of mdd group dataset is:", data_mdd.shape)
print("The size of bipolar type i group dataset is:", data_bpi.shape)
print("The size of bipolar type ii group dataset is:", data_bpii.shape)

--------------

----------

## 1D VAE model

> Checking data_no_s8 dataset, it requires balance for lacking data group \
> underneath, class is labeled 0(control), 1(mdd), 2(bpi), 3(bpii) \
> the ratio should be 1.00 * (control), 1.29 * (mdd), 3.04 * (bpi), 1.13 * (bpii)

In [ ]:
# data = data_fill_mean
# data = data_control
# data = data_mdd
# data = data_bpi
data = data_bpii

In [ ]:
data.shape

In [ ]:
class Args:
    # arugments
    epochs=150
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 4
    latent_dim = 16 ## need to find optimal latent size
    inter_dim1 = 128
    inter_dim2 = 64
    seed=710674

args = Args()

In [ ]:
data.columns

In [ ]:
### Using MDD,BP dataset
drop_list = ['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info', 'marriage_info', 'job_info']
x = data.drop(drop_list, axis=1)
y = data.loc[:,["target_info"]]

In [ ]:
x = x.fillna(x.mean())

In [ ]:
### MDD, BP ####### FULL GROUP CLASSIFICATION ##########

label = y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

data_y = to_categorical(label, 4) ## into the format of one-hot encoding

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=3)

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)

### encoder networks

In [ ]:
## deeper model
input_dim = data_x.shape[1]
latent_dim = args.latent_dim

inputs = Input(shape=(input_dim,))
h = Dense(128, activation='relu')(inputs)
h = Dense(64, activation='relu')(h)
h = Dense(32, activation='relu')(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [ ]:
## latent space sampling
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

### decoder networks

In [ ]:
## deeper model
decoder_h1 = Dense(32, activation='relu')
decoder_h2 = Dense(64, activation='relu')
decoder_h3 = Dense(128, activation='relu')
decoder_mean = Dense(input_dim, activation='sigmoid')

h_decoded = decoder_h1(z)
h_decoded = decoder_h2(h_decoded)
h_decoded = decoder_h3(h_decoded)
x_decoded_mean = decoder_mean(h_decoded)

### model define

In [ ]:
vae = Model(inputs, x_decoded_mean)
vae.summary()

### loss function

In [ ]:
reconstruction_loss = MeanSquaredError()(inputs, x_decoded_mean)

kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var + K.epsilon()), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)

In [ ]:
vae.compile(optimizer=Adam(learning_rate=0.001))

In [ ]:
# vae.summary()

### model training

In [ ]:
vae.fit(data_x, epochs=args.epochs, batch_size=args.bs, validation_split=0.2, verbose=2)

### extracting encoder, decoder

In [ ]:
## encoder model extraction
encoder = Model(inputs, z_mean)

In [ ]:
#### deeper model
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h1(decoder_input)
_h_decoded = decoder_h2(_h_decoded)
_h_decoded = decoder_h3(_h_decoded)
_x_decoded_mean = decoder_mean(_h_decoded)
decoder = Model(decoder_input, _x_decoded_mean)

### Latent space visualization

In [ ]:
# ## encode the input dataset into latent space
# encoded_data = encoder.predict(data_x)

# ## latent space transform with t-SNE function
# tsne = TSNE(n_components=2, random_state=710674)
# encoded_data_tsne = tsne.fit_transform(encoded_data)

# ## visualize
# plt.figure(figsize=(8, 6))
# plt.scatter(encoded_data_tsne[:, 0], encoded_data_tsne[:, 1], c=label, cmap='viridis')
# plt.colorbar()
# plt.xlabel("t-SNE component 1")
# plt.ylabel("t-SNE component 2")
# plt.title("t-SNE visualization of the latent space")
# plt.show()

### Synthetic data generation

> Our dataset contains 828 control, 457 mdd, 179 bp_i, 576 bp_ii \
> set the target number as 1000 \
> So, augment should be 172 control, 543 mdd, 821 bp_i, 424 bp_ii

In [ ]:
### sampling randomly from latent space
# n_samples = 172  # for control gen
# n_samples = 543  # for mdd gen
# n_samples = 821  # for bpi gen
n_samples = 424  # for bpii gen

z_samples = np.random.normal(size=(n_samples, latent_dim))

### 디코더를 통해 synthetic data 생성
# synthetic_data_con = decoder.predict(z_samples)
# synthetic_data_mdd = decoder.predict(z_samples)
# synthetic_data_bpi = decoder.predict(z_samples)
synthetic_data_bpii = decoder.predict(z_samples)

In [ ]:
# synthetic_data_con.shape
# synthetic_data_mdd.shape
# synthetic_data_bpi.shape
synthetic_data_bpii.shape

In [ ]:
#####################
# gen_con = synthetic_data_con.copy()
# gen_con = pd.DataFrame(gen_con, columns=data_x.columns)

# #####################
# gen_mdd = synthetic_data_mdd.copy()
# gen_mdd = pd.DataFrame(gen_mdd, columns=data_x.columns)

# #####################
# gen_bpi = synthetic_data_bpi.copy()
# gen_bpi = pd.DataFrame(gen_bpi, columns=data_x.columns)

####################
gen_bpii = synthetic_data_bpii.copy()
gen_bpii = pd.DataFrame(gen_bpii, columns=data_x.columns)

In [ ]:
gen_mdd.shape

----------

-----------

## Latent Diffusion Model

### encoder, decoder networks

In [ ]:
# Sampling function for the latent space
def sampling(repara):
    z_mean, z_log_var = repara
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Define encoder
def build_encoder(input_shape, latent_dim):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 3, activation='relu', padding='same')(inputs)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    z_mean = Dense(latent_dim)(x)
    z_log_var = Dense(latent_dim)(x)
    return Model(inputs, [z_mean, z_log_var], name='encoder')

# Define decoder
def build_decoder(output_shape, latent_dim):
    latent_inputs = Input(shape=(latent_dim,))
    x = Dense(128, activation='relu')(latent_inputs)
    x = Dense(np.prod(output_shape), activation='relu')(x)
    x = Reshape(output_shape)(x)
    outputs = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
    return Model(latent_inputs, outputs, name='decoder')

In [ ]:
input_shape = (x_train.shape[1], 1)  # Example input shape
latent_dim = 16

In [ ]:
## Define the VAE
encoder = build_encoder(input_shape, latent_dim)
decoder = build_decoder(input_shape, latent_dim)

inputs = Input(shape=input_shape)
z_mean, z_log_var = encoder(inputs)
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
outputs = decoder(z)

vae = Model(inputs, outputs, name='vae')

# Define the VAE loss
reconstruction_loss = MeanSquaredError()(inputs, outputs)
kl_loss = -0.5 * tf.reduce_mean(
    z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae_loss = reconstruction_loss + kl_loss
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

vae.summary()

In [ ]:
encoder.summary()

In [ ]:
decoder.summary()

### model training

In [ ]:
# Train the VAE
vae.fit(x_train, epochs=args.epochs, batch_size=args.bs, verbose=2)

### practice on new dataset

In [ ]:
vae_prediction = vae.predict(x_train[:])

In [ ]:
vae_prediction.shape

In [ ]:
vae_prediction_re = vae_prediction.reshape(1729,356)

In [ ]:
vae_prediction_re.shape

---------

-----------

## DNN with generated dataset

### using original dataset only

In [ ]:
data = data_fill_mean

In [ ]:
### Using MDD,BP dataset
drop_list = ['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info', 'marriage_info', 'job_info']
x = data.drop(drop_list, axis=1)
y = data.loc[:,["target_info"]]

In [ ]:
x = x.fillna(x.mean())

In [ ]:
### MDD, BP ####### FULL GROUP CLASSIFICATION ##########

label = y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

data_y = to_categorical(label, 4) ## into the format of one-hot encoding

In [ ]:
data_x = x.copy()

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_x[:] = scaler.fit_transform(data_x[:])
data_x = data_x.round(decimals=3)

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state = 710674)

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.1, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

***

### using original + generated dataset

> we generate synthesized dataset using VAE from above. \
> adopted original datset: data_mdd, data_bpi, data_bpii \
> synthesized into: gen_mdd, gen_bpi, gen_bpii \
> gen_control is not generated since "control group" has biggest number.

* Set target information into newly generated dataset

In [ ]:
gen_con['target_info'] = "control"
gen_mdd['target_info'] = "mdd"
gen_bpi['target_info'] = "bpi"
gen_bpii['target_info'] = "bpii"

In [ ]:
drop_list = ['subject_info', 'birth_info', 'sex_info', 'age_info', 'family_hx_info', 'marriage_info', 'job_info']
data_control = data_control.drop(drop_list, axis=1)
data_mdd = data_mdd.drop(drop_list, axis=1)
data_bpi = data_bpi.drop(drop_list, axis=1)
data_bpii = data_bpii.drop(drop_list, axis=1)

* Original dataframe concat

In [ ]:
ori_df_list = [data_control, data_mdd, data_bpi, data_bpii]
ori_df_concat = pd.concat(ori_df_list, ignore_index=True)

In [ ]:
ori_df_concat.shape

In [ ]:
ori_df_concat['target_info'].value_counts()

In [ ]:
gen_df_list = [gen_con, gen_mdd, gen_bpi, gen_bpii]
gen_df_concat = pd.concat(gen_df_list, ignore_index=True)

In [ ]:
gen_df_concat['target_info'].value_counts()

In [ ]:
### Using MDD,BP dataset
ori_x = ori_df_concat.drop(['target_info'], axis=1)
ori_y = ori_df_concat.loc[:,["target_info"]]

gen_x = gen_df_concat.drop(['target_info'], axis=1)
gen_y = gen_df_concat.loc[:,["target_info"]]

In [ ]:
ori_x = ori_x.fillna(ori_x.mean())

* Relabeling both(original, augmented) dataset into one-hot encoding

In [ ]:
label_ = ori_y
label_ = label_.replace({'control':0})
label_ = label_.replace({'mdd': 1})
label_ = label_.replace({'bpi': 2})
label_ = label_.replace({'bpii': 3})

y_ori = to_categorical(label_, 4) ## into the format of one-hot encoding

In [ ]:
label = gen_y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

y_gen = to_categorical(label, 4) ## into the format of one-hot encoding

* train-test dataset split

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(ori_x, y_ori, test_size = 0.4, random_state = 710674)

In [ ]:
x_train_concat = pd.concat([x_trainset, gen_x], ignore_index=True)
y_train_concat = np.concatenate([y_trainset, y_gen])

* train-validation dataset split

In [ ]:
x_train, x_vali, y_train, y_vali = train_test_split(x_train_concat, y_train_concat, test_size = 0.2, random_state = 710674)

* generating input dataset

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

### model training + test

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 4
    verbose='store_true'
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)
# kfold = StratifiedKFold(n_splits = split_num, shuffle = True)

In [ ]:
data_fill_mean['target_info'].value_counts()

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
# class_weight = {1:1.29, 2:3.04, 3: 1.13, 0:1}
class_weight = {0:1, 1:1.81, 2:4.62, 3: 1.43}
# class_weight = {0:1, 1:1, 2:1, 3: 1}

############## FOR NO CONTROL CLASSIFICATION ###############
# class_weight = {0: 1.4, 1: 2.74, 2: 1}

############## FOR NO BPII CLASSIFICATION ###############
# class_weight = {0: 1, 1: 1.29, 2: 3.04}

############## FOR BINARY CLASSIFICATION ###############
# class_weight = {0: 2.38, 1: 1}


In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
x_train.shape

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(256, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) #drop out
    model.add(Dense(64, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("=============================================")

***

***

## Using VAE/AE as a dimensionality reduction

In [ ]:
# data = data_no_s8
data = data_fill_s8

In [ ]:
data.shape

In [ ]:
data[data.isna().any(axis=1)].shape

In [ ]:
### Using MDD,BP dataset
data_x = data.drop(['subject_info', 'birth_info', 'target_info', 'sex_info', 'age_info', 'family_hx_info'], axis=1)
data_y = data.loc[:,["target_info"]]

In [ ]:
data_x = data_x.fillna(data_x.mean())

In [ ]:
### MDD, BP ####### FULL GROUP CLASSIFICATION ##########
x = data_x

label = data_y
label = label.replace({'control':0})
label = label.replace({'mdd': 1})
label = label.replace({'bpi': 2})
label = label.replace({'bpii': 3})

y = to_categorical(label, 4) ## into the format of one-hot encoding

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=32
    lr=0.001
    momentum=0.9
    encoding_dim = 32
    num_classes= 4
    verbose='store_true'
    seed=710674

args = Args()

In [ ]:
x.shape

### Autoencoder for dim reduction

In [ ]:
# input dataset layer
input_layer = Input(shape=(x.shape[1],))

# encoder layers
encoder = Dense(256, activation='relu')(input_layer)
encoder = Dense(128, activation='relu')(encoder)
encoder_out = Dense(args.encoding_dim, activation='relu')(encoder)

# decoder layers
decoder = Dense(128, activation='relu')(encoder_out)
decoder = Dense(256, activation='relu')(decoder)
decoder_out = Dense(x.shape[1], activation='sigmoid')(decoder)

In [ ]:
# AE model
autoencoder = Model(inputs=input_layer, outputs=decoder_out)

# Encoder model (convert input dataset into latent space)
encoder_model = Model(inputs=input_layer, outputs=encoder_out)

# Decoder model (recover latent space/vector into original dataset format)
encoded_input = Input(shape=(args.encoding_dim,))
decoder_layer = autoencoder.layers[-3](encoded_input)
decoder_layer = autoencoder.layers[-2](decoder_layer)
decoder_out = autoencoder.layers[-1](decoder_layer)
decoder_model = Model(inputs=encoded_input, outputs=decoder_layer)

In [ ]:
# Model compile
autoencoder.compile(optimizer='adam', loss='mse')

# Checking each model
autoencoder.summary()
# encoder_model.summary()
# decoder_model.summary()

In [ ]:
## Model training
autoencoder.fit(x, x, epochs = args.epochs, batch_size = args.bs, shuffle=True, validation_split=0.2, verbose=2)

In [ ]:
encoded_data = encoder_model.predict(x)
decoded_data = decoder_model.predict(encoded_data)

In [ ]:
encoded_data.shape

In [ ]:
decoded_data.shape

In [ ]:
y.shape

In [ ]:
class Args:
    # arugments
    epochs=200
    bs=32
    lr=0.001
    momentum=0.9
    num_classes= 4
    verbose='store_true'
    seed=710674

args = Args()

# np.random.seed(args.seed)
# random.seed(args.seed)
# torch.manual_seed(args.seed)

In [ ]:
x_trainset, x_test, y_trainset, y_test = train_test_split(decoded_data, y, test_size = 0.1, random_state = 710674)
x_train, x_vali, y_train, y_vali = train_test_split(x_trainset, y_trainset, test_size = 0.2, random_state = 710674)

In [ ]:
inputs = np.concatenate((x_train, x_vali), axis = 0)
targets = np.concatenate((y_train, y_vali), axis = 0)

In [ ]:
fold_num = 1
split_num = 5
opt = keras.optimizers.SGD(learning_rate = args.lr, decay = 1e-6, momentum = args.momentum)
kfold = KFold(n_splits = split_num, shuffle = True)
# kfold = StratifiedKFold(n_splits = split_num, shuffle = True)

In [ ]:
############## FOR FOUR-GROUP CLASSIFICATION ###############
class_weight = {1:1.29, 2:3.04, 3: 1.13, 0:1}

In [ ]:
acc_per_fold = []
loss_per_fold = []

In [ ]:
for train, test in kfold.split(inputs, targets):
    model = Sequential()
    model.add(Dense(128, input_dim = x_train.shape[1], activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5)) #drop out
    model.add(Dense(32, activation = 'relu'))
    model.add(Dense(args.num_classes, activation = 'softmax'))
    
    ## model compile
    model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])
    
    print('----------------------------------------')
    print(f'Training or fold {fold_num} ... ')
    
    ## fit data to model
    history = model.fit(inputs[train], targets[train], batch_size = args.bs, epochs = args.epochs, verbose = 0, class_weight = class_weight)
    
    ## generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test])
    print(f'Score for fold {fold_num}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    ## increasing fold number
    fold_num = fold_num + 1
    
    
    
## Summarizing the results
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'>> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'>>> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'>>> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
y_predict = model.predict(x_test)
y_predict = np.argmax(y_predict, axis = 1)
y_test = np.argmax(y_test, axis = 1)

result = confusion_matrix(y_test, y_predict, normalize = 'pred')
print(result)

In [ ]:
figure = plt.figure(figsize=(6, 4))
sns.heatmap(result, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
accuracy = metrics.accuracy_score(y_test, y_predict)
precision = metrics.precision_score(y_test, y_predict, average = 'macro')
recall = metrics.recall_score(y_test, y_predict, average = 'micro')
f1 = metrics.f1_score(y_test, y_predict, average = 'weighted')

print("=============================================")
print("The overall accuracy is:", round(accuracy, 4))
print("The precision score is:", round(precision, 4))
print("The recall score is:", round(recall, 4))
print("The f1 score is:", round(f1, 4))
print("=============================================")